In [2]:
import os
import xlrd
import collections
import re
import xlsxwriter 
import datetime
import math

In [3]:
half_list = [u"%dH%d"%(y_, h_) for y_ in xrange(2004,2017) for h_ in xrange(1,3)]
half_date_list = [datetime.datetime(year=y, month=m, day=1)  for y in xrange(2004, 2018) for m in (1,7)]
half_list.append(u"2017H1")
symbol_ro = re.compile('^\d{6}$')
hy_ro = re.compile('^\d{4}H[1|2]$')

In [20]:
funds_excel = xlrd.open_workbook('fund_stock_holding_test.xlsx').sheet_by_index(0)

In [3]:
rtn_dict2 = lambda : collections.defaultdict(dict)
rtn_dict1 = lambda : collections.defaultdict(rtn_dict2)

In [22]:
funds_data = collections.defaultdict(rtn_dict2)
funds_disclusure_data = collections.defaultdict(lambda :collections.defaultdict(set))
for i in xrange(3, funds_excel.nrows):
    funds_no = funds_excel.cell(i, 0).value
    funds_hy = funds_excel.cell(i, 2).value
    funds_symbol = funds_excel.cell(i, 6).value
    funds_shares = funds_excel.cell(i, 8).value
    if symbol_ro.match(funds_no) and symbol_ro.match(funds_symbol) and hy_ro.match(funds_hy) and funds_shares:
        funds_data[funds_no][funds_symbol][funds_hy] = float(funds_shares)
        funds_disclusure_data[funds_no][funds_hy].add(funds_symbol)
    else:
        print u"第%d行的格式有问题，%s,%s,%s,%s"%(i, (funds_no), (funds_hy), (funds_symbol), unicode(funds_shares))
print "共加载%d只基金的数据"%len(funds_data)

共加载936只基金的数据


In [4]:
def hy_add(hy_):
    year_ = int(hy_[0:4])
    half_ = int(hy_[-1])
    if half_ == 1:
        return u"%dH%d"%(year_, 2)
    else:
        return u"%dH%d"%(year_+1,1)

def hy_minus(hy_):
    year_ = int(hy_[0:4])
    half_ = int(hy_[-1])
    if half_ == 1:
        return u"%dH%d"%(year_-1, 2) 
    else:
        return u"%dH%d"%(year_, 1)
    
def hy_2_datetime(hy_):
    year_ = int(hy_[0:4])
    half_ = int(hy_[-1])
    if half_ == 1:
        return datetime.datetime(year=year_, month=1, day=1)
    else:
        return datetime.datetime(year=year_, month=7, day=1)
    
print hy_add("2018H1")
print hy_add("2018H2")
print hy_minus("2018H1")
print hy_minus("2018H2")
print hy_2_datetime("2018H1")
print hy_2_datetime("2018H2")

2018H2
2019H1
2017H2
2018H1
2018-01-01 00:00:00
2018-07-01 00:00:00


In [27]:
def calc_share_change(data_):
    rtn_ = {}
    for idx_, hy_ in enumerate(half_list):
        pre_hy_ = hy_minus(hy_)
        if pre_hy_ in data_ and hy_ in data_:
            if data_[hy_] > data_[pre_hy_]:
                action_ = u"buy"
            elif data_[hy_] < data_[pre_hy_]:
                action_ = u"sold"
            else:
                action_ = u"nothing"
            rtn_[hy_] = ((data_[hy_] - data_[pre_hy_]) / data_[pre_hy_], action_)
        elif pre_hy_ not in data_ and hy_ in data_:
            rtn_[hy_] = ((data_[hy_] - 1), u"startup")
        elif pre_hy_ in data_ and hy_ not in data_:
            rtn_[hy_] = (-1.0, u"clearup")
        if pre_hy_ in data_ and hy_ in data_ and ((data_[hy_] / data_[pre_hy_]) > 10 or data_[pre_hy_] <= 100):
            # 如果前期小于100股，或者后期/前期>100，认为是个startup
            rtn_[hy_] = ((data_[hy_] - 1), u"startup")
    return rtn_

def calc_share_chage_log(date_):
    rtn_ = {}
    for idx_, hy_ in enumerate(half_list):
        pre_hy_ = hy_minus(hy_)
        if pre_hy_ in data_ and hy_ in data_:
            if data_[hy_] > data_[pre_hy_]:
                action_ = u"buy"
            elif data_[hy_] < data_[pre_hy_]:
                action_ = u"sold"
            else:
                action_ = u"nothing"
            rtn_[hy_] = (math.log(data_[hy_]) - math.log(data_[pre_hy_]), action_)
        elif pre_hy_ not in data_ and hy_ in data_:
            rtn_[hy_] = (math.log(data_[hy_]), u"startup")
        elif pre_hy_ in data_ and hy_ not in data_:
            rtn_[hy_] = (-math.log(data_[pre_hy_]), u"clearup")
    return rtn_

if os.path.exists(u"funds_shares_change.xlsx"):
    os.remove(u"funds_shares_change.xlsx")

workbook = xlsxwriter.Workbook(u'funds_shares_change.xlsx')
worksheet = workbook.add_worksheet()

row_idx = 0
for funds_no in funds_data:
    for funds_symbol in funds_data[funds_no]:
        share_change = calc_share_change(funds_data[funds_no][funds_symbol])
        for hy in sorted(share_change.keys()):
            rate, action = share_change[hy]
            worksheet.write(row_idx, 0, funds_no)
            worksheet.write(row_idx, 1, funds_symbol)
            worksheet.write(row_idx, 2, hy)
            worksheet.write(row_idx, 3, action)
            worksheet.write(row_idx, 4, rate)
            row_idx+=1
            
workbook.close()

In [14]:
print funds_data[u'000828'][u'000411']

defaultdict(<type 'dict'>, {u'2005H1': {}, u'2013H1': {}, u'2013H2': {}, u'2005H2': {}, u'2011H2': {}, u'2011H1': {}, u'2010H2': {}, u'2010H1': {}, u'2008H1': {}, u'2008H2': {}, u'2009H1': {}, u'2009H2': {}, u'2015H2': 78400.0, u'2015H1': {}, u'2014H2': {}, u'2014H1': {}, u'2012H1': {}, u'2012H2': {}, u'2007H2': {}, u'2007H1': {}, u'2004H1': {}, u'2004H2': {}, u'2003H2': {}, u'2006H2': {}, u'2006H1': {}})


In [8]:
if os.path.exists(u"funds_Disclosure.xlsx"):
    os.remove(u"funds_Disclosure.xlsx")

workbook = xlsxwriter.Workbook(u'funds_Disclosure.xlsx')
worksheet = workbook.add_worksheet()

row_idx = 0
for funds_no in funds_disclusure_data:
    for hy in sorted(funds_disclusure_data[funds_no].keys()):
        worksheet.write(row_idx, 0, funds_no)
        worksheet.write(row_idx, 1, hy)
        worksheet.write(row_idx, 2, len(funds_disclusure_data[funds_no][hy]))
        row_idx += 1

In [9]:
if os.path.exists(u"funds_shares_change1.xlsx"):
    os.remove(u"funds_shares_change1.xlsx")

workbook = xlsxwriter.Workbook(u'funds_shares_change1.xlsx')
worksheet = workbook.add_worksheet()

row_idx = 0
for funds_no in funds_data:
    for funds_symbol in funds_data[funds_no]:
        share_change = calc_share_change(funds_data[funds_no][funds_symbol])
        for hy in sorted(share_change.keys()):
            rate, action = share_change[hy]
            if action in ("buy", "sold", "nothing"):
                worksheet.write(row_idx, 0, funds_no)
                worksheet.write(row_idx, 1, funds_symbol)
                worksheet.write(row_idx, 2, hy)
                worksheet.write(row_idx, 3, action)
                worksheet.write(row_idx, 4, rate)
                row_idx+=1
            
workbook.close()

In [15]:
sheet4=xlrd.open_workbook('herding tendency caculation.xlsx').sheet_by_index(0)

d4 = collections.defaultdict(dict)
d4_hending = collections.defaultdict(dict)

for i in xrange(1, sheet4.nrows):
    symbol = sheet4.cell(i,1).value
    hy = sheet4.cell(i,2).value
    if symbol_ro.match(symbol) and hy_ro.match(hy) \
        and sheet4.cell(i,3).ctype==2 and sheet4.cell(i,4).ctype==2 and sheet4.cell(i,5).ctype==2 and sheet4.cell(i,6).ctype==2:
        d4[symbol][hy] = [sheet4.cell(i,3).value,sheet4.cell(i,4).value,sheet4.cell(i,5).value,sheet4.cell(i,6).value,sheet4.cell(i,0).value]
        d4_hending[symbol][hy] = sheet4.cell(i,3).value



In [20]:
for symbol in d4:
    for idx_, hy_ in enumerate(half_list):
        if hy_ not in d4[symbol]:
            continue
        pre_hy_ = hy_minus(hy_)
        pre_pre_hy_ = hy_minus(pre_hy_)
        
        fengzi_ = d4_hending[symbol][hy_]
        fengmu_ = 1.0
        counter = 2.0
        
        while pre_hy_ in d4_hending[symbol]:
            fengzi_ += (1/counter) * d4_hending[symbol][pre_hy_]
            fengmu_ += (1/counter)
            
            pre_hy_ = hy_minus(pre_hy_)
            counter += 1
            
        d4[symbol][hy_][0] = fengzi_ / fengmu_

In [22]:
if os.path.exists(u"herding tendency caculation1.xlsx"):
    os.remove(u"herding tendency caculation1.xlsx")

workbook = xlsxwriter.Workbook(u'herding tendency caculation1.xlsx')
worksheet = workbook.add_worksheet() 

wt_idx = 0
for symbol in d4:
    for hy_ in d4[symbol]:
        worksheet.write(wt_idx, 0, d4[symbol][hy_][4])
        worksheet.write(wt_idx, 1, symbol)
        worksheet.write(wt_idx, 2, hy_)
        worksheet.write(wt_idx, 3, d4[symbol][hy_][0])
        worksheet.write(wt_idx, 4, d4[symbol][hy_][1])
        worksheet.write(wt_idx, 5, d4[symbol][hy_][2])
        worksheet.write(wt_idx, 6, d4[symbol][hy_][3])
        worksheet.write(wt_idx, 7, d4_hending[symbol][hy_])
        wt_idx += 1

workbook.close()

print "done"

done


In [23]:
sheet5=xlrd.open_workbook('herding tendency caculation_standardize.xlsx').sheet_by_index(0)

d5 = collections.defaultdict(dict)
d5_hending = collections.defaultdict(dict)

for i in xrange(1, sheet5.nrows):
    symbol = sheet5.cell(i,1).value
    hy = sheet5.cell(i,2).value
    if symbol_ro.match(symbol) and hy_ro.match(hy) \
        and sheet5.cell(i,3).ctype==2 and sheet5.cell(i,4).ctype==2 and \
        sheet5.cell(i,5).ctype==2 and sheet5.cell(i,6).ctype==2 and sheet5.cell(i,7).ctype==2:
        d5[symbol][hy] = [sheet5.cell(i,3).value,sheet5.cell(i,4).value,sheet5.cell(i,5).value,\
                          sheet5.cell(i,6).value,sheet5.cell(i,7).value,sheet5.cell(i,0).value]
        d5_hending[symbol][hy] = sheet5.cell(i,3).value


for symbol in d5:
    for idx_, hy_ in enumerate(half_list):
        if hy_ not in d5[symbol]:
            continue
        pre_hy_ = hy_minus(hy_)
        pre_pre_hy_ = hy_minus(pre_hy_)
        
        fengzi_ = d5_hending[symbol][hy_]
        fengmu_ = 1.0
        counter = 2.0
        
        while pre_hy_ in d5_hending[symbol]:
            fengzi_ += (1/counter) * d5_hending[symbol][pre_hy_]
            fengmu_ += (1/counter)
            
            pre_hy_ = hy_minus(pre_hy_)
            counter += 1
            
        d5[symbol][hy_][0] = fengzi_ / fengmu_
        
if os.path.exists(u"herding tendency caculation_standardize1.xlsx"):
    os.remove(u"herding tendency caculation_standardize1.xlsx")

workbook = xlsxwriter.Workbook(u'herding tendency caculation_standardize1.xlsx')
worksheet = workbook.add_worksheet() 

wt_idx = 0
for symbol in d5:
    for hy_ in d5[symbol]:
        worksheet.write(wt_idx, 0, d5[symbol][hy_][5])
        worksheet.write(wt_idx, 1, symbol)
        worksheet.write(wt_idx, 2, hy_)
        worksheet.write(wt_idx, 3, d5[symbol][hy_][0])
        worksheet.write(wt_idx, 4, d5[symbol][hy_][1])
        worksheet.write(wt_idx, 5, d5[symbol][hy_][2])
        worksheet.write(wt_idx, 6, d5[symbol][hy_][3])
        worksheet.write(wt_idx, 7, d5[symbol][hy_][4])
        worksheet.write(wt_idx, 8, d5_hending[symbol][hy_])
        wt_idx += 1

workbook.close()

print "done"

done


In [5]:
jingli_excel = xlrd.open_workbook(u'基金经理信息.xlsx').sheet_by_index(0)
jingli_info = {}
for i in xrange(1, jingli_excel.nrows):
    jingli_info[jingli_excel.cell(i, 0).value] = jingli_excel.row(i)

In [6]:
symbol1_ro = re.compile(u"^(\d{6})\.OF$")

jingli_jijin_tmp_info = collections.defaultdict(list)
jingli_jijin_start_date = collections.defaultdict(lambda : collections.defaultdict(list))

jingli_jijin_excel = xlrd.open_workbook(u'基金历任基金经理明细表.xlsx').sheet_by_index(0)
for i in xrange(1, jingli_jijin_excel.nrows):
    symbol = symbol1_ro.match(jingli_jijin_excel.cell(i, 0).value).groups()[0]
    name = jingli_jijin_excel.cell(i, 5).value
    start_date = datetime.datetime.strptime(jingli_jijin_excel.cell(i, 7).value, "%Y-%m-%d")
    if jingli_jijin_excel.cell(i, 8).value:
        end_date = datetime.datetime.strptime(jingli_jijin_excel.cell(i, 8).value, "%Y-%m-%d")
    else:
        end_date = datetime.datetime.today()
    jingli_jijin_tmp_info[symbol].append((name,  start_date, end_date))
    jingli_jijin_start_date[symbol][name].append(start_date)
            

In [41]:
debug_symbol = u'002386'
debug_hy = u'2017H1'

jingli_jijin_info = collections.defaultdict(dict)
for symbol in jingli_jijin_tmp_info:
    for i in xrange(len(half_date_list)-1):
        hy = u"%dH%d"%(half_date_list[i].year, half_date_list[i].month/7+1)
        flag_list = [1 if (jingli[1] <= half_date_list[i] and (half_date_list[i+1] - datetime.timedelta(days=1)) <= jingli[2]) else 0 \
                     for jingli in jingli_jijin_tmp_info[symbol]]
        flag_list_left = [1 if (jingli[1] <= half_date_list[i] \
                                and (half_date_list[i+1] - datetime.timedelta(days=1)) > jingli[2] and jingli[2] >= half_date_list[i]) else 0 \
                          for jingli in jingli_jijin_tmp_info[symbol]]
        flag_list_right = [1 if (jingli[1] > half_date_list[i] and jingli[1] <= (half_date_list[i+1] - datetime.timedelta(days=1)) \
                                 and (half_date_list[i+1] - datetime.timedelta(days=1)) <= jingli[2]) else 0 \
                           for jingli in jingli_jijin_tmp_info[symbol]]
        if symbol == debug_symbol and hy == debug_hy:
            print flag_list
            print flag_list_left
            print flag_list_right
        if sum(flag_list) == 1:
            # 只有一个经理全覆盖半年
            this_name = [jingli_jijin_tmp_info[symbol][i][0] for i in xrange(len(flag_list)) if flag_list[i] == 1]
            jingli_jijin_info[symbol][hy] = this_name[0]
        elif sum(flag_list) == 0:
            # 没有经理全覆盖半年
            if sum(flag_list_left) + sum(flag_list_right) == 1:
                # 只有一个经理部分覆盖半年
                this_name = [jingli_jijin_tmp_info[symbol][i][0] for i in xrange(len(flag_list_left)) if flag_list_left[i] == 1] \
                     +[jingli_jijin_tmp_info[symbol][i][0] for i in xrange(len(flag_list_right)) if flag_list_right[i] == 1]
                jingli_jijin_info[symbol][hy] = this_name[0]
            elif sum(flag_list_left) + sum(flag_list_right) == 0:
                # 没有经理部分覆盖半年
                continue
            else:
                # 多个经理部分覆盖半年
                # 首先看覆盖后边界的，先选个最先的，如果在后边界前60天就参与了，那就是他
                min_date = half_date_list[i+1]
                min_name = u"多个经理部分覆盖后边界"
                for i_, jingli in enumerate(jingli_jijin_tmp_info[symbol]):
                    if jingli[1] <= min_date and flag_list_right[i_]:
                        if symbol == debug_symbol and hy == debug_hy:
                            print min_name, min_date
                        min_name = jingli[0]
                        min_date = jingli[1]
                # 没有覆盖前半界的，或者后半界参与超过60天
                if sum(flag_list_left) == 0 or min_date < (half_date_list[i+1] - datetime.timedelta(days=60)):
                    jingli_jijin_info[symbol][hy] = min_name              
                else: 
                    # 然后看覆盖前边界，选个最后的
                    max_date = half_date_list[i]
                    max_name = u"多个经理部分覆盖前边界"
                    for i_, jingli in enumerate(jingli_jijin_tmp_info[symbol]):
                        if jingli[2] >= max_date and flag_list_left[i_]:
                            if symbol == debug_symbol and hy == debug_hy:
                                print max_name, max_date
                            max_name = jingli[0]
                            max_date = jingli[2]
                    jingli_jijin_info[symbol][hy] = max_name
        else:
            # 多个经理全覆盖半年，选择最早的那个
            min_date = datetime.datetime.today()
            min_name = u"多个经理部分覆盖全半年"
            for i_, jingli in enumerate(jingli_jijin_tmp_info[symbol]):
                if jingli[1] <= min_date and flag_list[i_] == 1:
                    min_name = jingli[0]
                    min_date = jingli[1]
            jingli_jijin_info[symbol][hy] = min_name

In [42]:
for hy in sorted(jingli_jijin_info[debug_symbol].keys()):
    print hy, jingli_jijin_info[debug_symbol][hy]

In [38]:
if os.path.exists(u"经理信息.xlsx"):
    os.remove(u"经理信息.xlsx")

workbook = xlsxwriter.Workbook(u'经理信息.xlsx')
worksheet = workbook.add_worksheet() 

wt_idx = 0
for symbol_ in jingli_jijin_info:
    for hy_ in jingli_jijin_info[symbol_]:
        hy_left_ = hy_2_datetime(hy_)
        hy_right_ = hy_2_datetime(hy_add(hy_)) - datetime.timedelta(days=1)
        name_ = jingli_jijin_info[symbol_][hy_]
        if name_ in jingli_info:
            worksheet.write(wt_idx, 0, symbol_)
            worksheet.write(wt_idx, 1, hy_)
            worksheet.write(wt_idx, 2, name_)
            worksheet.write(wt_idx, 3, jingli_info[name_][1].value)
            worksheet.write(wt_idx, 4, jingli_info[name_][2].value)
            worksheet.write(wt_idx, 5, jingli_info[name_][3].value)
            worksheet.write(wt_idx, 6, jingli_info[name_][4].value)
            
            
            for i_, start_date_ in enumerate(sorted(jingli_jijin_start_date[symbol_][name_])):
                if i_ == len(jingli_jijin_start_date[symbol_][name_]) - 1 or 
                worksheet.write(wt_idx, 7, jingli_jijin_start_date[symbol_][name_])
            wt_idx += 1
        else:
            print u"can't find %s symbol_ %s hy_ %s"%(name_, symbol_, hy_)
workbook.close()
            

In [ ]:
########################去除打新#############################
sheet_daxin=xlrd.open_workbook(u'减仓加仓.xlsx').sheet_by_index(0)

if os.path.exists(u"减仓加仓1.xlsx"):
    os.remove(u"减仓加仓1.xlsx")

workbook = xlsxwriter.Workbook(u'减仓加仓1.xlsx')
worksheet = workbook.add_worksheet() 

wt_idx = 0
for i in xrange(0, sheet_daxin.nrows):
    if i > 0:
        listing_date = datetime.datetime.strptime(sheet_daxin.cell(i, 3).value, "%Y-%m-%d")
        trading_date = sheet_daxin.cell(i, 4).value
        if listing_date >= datetime.datetime(year=2015, month=1, day=1):
            listing_hy = u"%dH%d"%(listing_date.year, listing_date.month/7+1)
            if listing_hy == hy_minus(trading_date):
                continue
    for j in xrange(0, sheet_daxin.ncols):
        worksheet.write(wt_idx, j, sheet_daxin.cell(i, j).value)
    wt_idx += 1
    
workbook.close()
#####################################################